The following code cretes a "good random CP matrix and team_strength(numbers[]) list"

In [17]:
import random

# Initialize variables
total = 0
numbers = []

# Generate team strength such that total strength is less than 2*900
while total <= 1800:
    num = random.randint(100, 150)
    if total + num > 1800:
        break
    numbers.append(num)
    total += num

print("Generated numbers:", numbers)
print("Total sum:", total)

import random

# Function to generate a random binary matrix with all 1's in the first row
def generate_binary_matrix_with_first_row_ones(rows, cols):
    matrix = [[random.randint(0, 1) for _ in range(cols)] for _ in range(rows)]
    # Set all elements of the first row to 1
    matrix[0] = [1] * cols
    return matrix

# Function to check if the condition holds
def check_condition(binary_matrix, numbers):
    mola_sums = []
    for col in range(len(binary_matrix[0])):
        mola_sum = sum(binary_matrix[row][col] * numbers[row] for row in range(len(numbers)))
        mola_sums.append(mola_sum)
    return all(mola_sum <= 900 for mola_sum in mola_sums)

# Keep generating binary matrices until the condition holds
while True:
    binary_matrix = generate_binary_matrix_with_first_row_ones(len(numbers), 5)
    if check_condition(binary_matrix, numbers):
        break

# Print the binary matrix
print("Generated binary matrix:")
for row in binary_matrix:
    print(row)

# Check the condition
mola_sums = []
for col in range(len(binary_matrix[0])):
    mola_sum = sum(binary_matrix[row][col] * numbers[row] for row in range(len(numbers)))
    mola_sums.append(mola_sum)

print("Mola sums:", mola_sums)
print("Condition holds:", all(mola_sum <= 900 for mola_sum in mola_sums))



CP=binary_matrix


Generated numbers: [119, 133, 100, 122, 133, 132, 122, 140, 126, 113, 126, 101, 114, 114]
Total sum: 1695
Generated binary matrix:
[1, 1, 1, 1, 1]
[1, 1, 1, 0, 0]
[1, 1, 0, 1, 1]
[0, 1, 1, 0, 1]
[0, 0, 1, 0, 1]
[0, 0, 0, 0, 1]
[1, 1, 0, 0, 0]
[0, 0, 1, 1, 0]
[1, 0, 1, 1, 1]
[0, 0, 0, 0, 0]
[0, 1, 0, 1, 0]
[0, 0, 0, 0, 1]
[1, 0, 0, 0, 0]
[0, 0, 1, 0, 0]
Mola sums: [714, 722, 887, 611, 833]
Condition holds: True


The following Created the gurobi Model, along with the objective function and Constraints  

In [33]:
import gurobipy as gp
from gurobipy import GRB

# Create a new model
model = gp.Model("CP_Problem")

# Define T and S
T = int(len(numbers) * 0.6)
S = len(numbers) - T

# Define the decision variables y_ijk
y = model.addVars(((i, j, k) for i in range(T+S+1) for j in range(1, 7) for k in range(1, 6)), vtype=GRB.BINARY, name="y")

# Define the decision variables X_ijk
X = model.addVars(((i, j, k) for i in range(T+S+1) for j in range(1, 7) for k in range(1, 6)), lb=0, ub=150, vtype=GRB.INTEGER, name="X")

# Define the decision variables y_new_jk
y_new = model.addVars(((j, k) for j in range(1, 7) for k in range(1, 6)), vtype=GRB.BINARY, name="y_new")

# Define the objective function
# Define the objective function
objective1 = 10 * gp.quicksum(y_new[j, k] for j in range(1, 7) for k in range(1, 6))
objective2 = gp.quicksum(y[i, j, k] for i in range(1, T+S) for j in range(1, 7) for k in range(1, 6) if CP[i][k-1] == 1)

# Combine both objectives into a single objective
total_objective = objective1 + objective2

# Set the objective in the model
model.setObjective(total_objective, GRB.MINIMIZE)


# Add constraints for the existing variables y
for i in range(T+S+1):
    for j in range(1, 7):
        for k in range(1, 6):
            model.addConstr(X[i, j, k] <= 900 * y[i, j, k])

# Add constraints X[I,j,k] >= 0 for all I, j, k
for i in range(T+S+1):
    for j in range(1, 7):
        for k in range(1, 6):
            model.addConstr(X[i, j, k] >= 0)

# Add constraints for the new variables y_new
for j0 in range(1, 7):
    for k0 in range(1, 6):
        model.addConstr(gp.quicksum(X[i, j0, k0] for i in range(T+S+1)) <= 900 * y_new[j0, k0])

# Add constraint 1: Sum of X_ijk <= 150 for all j, k
for j in range(1, 7):
    for k in range(1, 6):
        model.addConstr(gp.quicksum(X[i, j, k] for i in range(T+S+1)) <= 150)

# Add constraint 2: y_ijk = 0 for CP(I, k) = 0
for i in range(1, T+S):
    for j in range(1, 7):
        for k in range(1, 6):
            if CP[i][k-1] == 0:
                model.addConstr(y[i, j, k] == 0)

# Add constraint 3: Sum of y_ijk >= 1 for CP(I, k) = 1
for j in range(1, 7):
    for k in range(1, 6):
        model.addConstr(gp.quicksum(y[i, j, k] for i in range(1, T+S)) >= 1)

# Add constraint 4: Sum of X_ijk over j for all I and k such that CP[I][k-1]==1 = numbers[I]
for i in range(T+S):
    for k in range(1, 6):
        if CP[i][k-1] == 1:
            model.addConstr(gp.quicksum(X[i, j, k] for j in range(1, 7)) == numbers[i])

# Solve the problem
model.optimize()




Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[arm] - Darwin 22.1.0 22A400)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1255 rows, 930 columns and 3090 nonzeros
Model fingerprint: 0x44e62156
Variable types: 0 continuous, 930 integer (480 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+02]
  Objective range  [1e+00, 1e+01]
  Bounds range     [1e+00, 2e+02]
  RHS range        [1e+00, 2e+02]
Found heuristic solution: objective 336.0000000
Presolve removed 1200 rows and 846 columns
Presolve time: 0.03s
Presolved: 55 rows, 84 columns, 198 nonzeros
Found heuristic solution: objective 305.0000000
Variable types: 0 continuous, 84 integer (42 binary)

Root relaxation: objective 2.955333e+02, 62 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     

In [19]:
# Print the status of the solution
print("Status:", model.status)


Status: 2


The following gets total employees on each fllor and on each day

In [34]:
# Initialize a dictionary to store the sum of X[i, j, k] for each j and k
sum_X = {}

# Calculate the sum of X[i, j, k] for each j and k
for j in range(1, 7):
    for k in range(1, 6):
        sum_X[(j, k)] = sum(X[i, j, k].x for i in range(T+S+1) if X[i, j, k].x is not None)

# Print the sum of X[i, j, k] for each j and k
for j in range(1, 7):
    for k in range(1, 6):
        print("Sum of X for j =", j, "and k =", k, ":", sum_X[(j, k)] if sum_X[(j, k)] is not None else 0)


Sum of X for j = 1 and k = 1 : 150.0
Sum of X for j = 1 and k = 2 : 150.0
Sum of X for j = 1 and k = 3 : 150.0
Sum of X for j = 1 and k = 4 : 126.0
Sum of X for j = 1 and k = 5 : 150.0
Sum of X for j = 2 and k = 1 : 114.0
Sum of X for j = 2 and k = 2 : 150.0
Sum of X for j = 2 and k = 3 : 150.0
Sum of X for j = 2 and k = 4 : 100.0
Sum of X for j = 2 and k = 5 : 150.0
Sum of X for j = 3 and k = 1 : 150.0
Sum of X for j = 3 and k = 2 : 146.0
Sum of X for j = 3 and k = 3 : 150.0
Sum of X for j = 3 and k = 4 : 119.0
Sum of X for j = 3 and k = 5 : 146.0
Sum of X for j = 4 and k = 1 : 150.0
Sum of X for j = 4 and k = 2 : 0.0
Sum of X for j = 4 and k = 3 : 150.0
Sum of X for j = 4 and k = 4 : 0.0
Sum of X for j = 4 and k = 5 : 122.0
Sum of X for j = 5 and k = 1 : 150.0
Sum of X for j = 5 and k = 2 : 126.0
Sum of X for j = 5 and k = 3 : 137.0
Sum of X for j = 5 and k = 4 : 126.0
Sum of X for j = 5 and k = 5 : 133.0
Sum of X for j = 6 and k = 1 : 0.0
Sum of X for j = 6 and k = 2 : 150.0
Sum of 

In [38]:
# Initialize a dictionary to store the values of X[i, j, 1]
X_values_k1 = {}

# Collect values of X[i, j, 1] for each i
for i in range(T+S+1):
    X_values_k1[i] = [X[i, j, 2].x if X[i, j, 2].x is not None else 0 for j in range(1, 7)]

# Convert the dictionary to a matrix
X_matrix_k1 = [[X_values_k1[i][j] for j in range(6)] for i in range(T+S+1)]

# Print the matrix
for row in X_matrix_k1:
    print(row)


[28.0, 50.0, 24.0, 0.0, 0.0, 17.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 133.0]
[0.0, 100.0, 0.0, 0.0, 0.0, 0.0]
[122.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[-0.0, -0.0, -0.0, -0.0, -0.0, -0.0]
[-0.0, -0.0, -0.0, -0.0, -0.0, -0.0]
[0.0, 0.0, 122.0, 0.0, 0.0, 0.0]
[-0.0, -0.0, -0.0, -0.0, -0.0, -0.0]
[-0.0, -0.0, -0.0, -0.0, -0.0, -0.0]
[-0.0, -0.0, -0.0, -0.0, -0.0, -0.0]
[0.0, 0.0, 0.0, 0.0, 126.0, 0.0]
[-0.0, -0.0, -0.0, -0.0, -0.0, -0.0]
[-0.0, -0.0, -0.0, -0.0, -0.0, -0.0]
[-0.0, -0.0, -0.0, -0.0, -0.0, -0.0]
[-0.0, -0.0, -0.0, -0.0, -0.0, -0.0]


In [ ]:
# Print the values of all decision variables
for v in model.getVars():
    print(v.VarName, "=", v.x)
